# Evals API: 音声入力

このクックブックでは、音声ベースのタスクにOpenAIのEvalsフレームワークを使用する方法を説明します。Evals APIを活用して、音声メッセージとプロンプトに対するモデル生成応答を評価します。**サンプリング**を使用してモデル応答を生成し、**モデル評価**を使用して出力音声と参考回答に対してモデル応答をスコア化します。評価は、サンプリングされた応答からの音声出力に対して行われることに注意してください。

音声サポートが追加される前は、音声会話を評価するために、まずテキストに転写する必要がありました。現在では、元の音声を使用し、モデルからも音声でサンプルを取得できます。これにより、ユーザーとエージェントの両方が音声を使用するカスタマーサポートシナリオなどのワークフローをより正確に表現できます。評価では、音声モデルを使用してモデル評価者で音声応答を評価します。代替案として、またはそれと組み合わせて、サンプリングされた音声からのテキスト転写を使用し、既存のテキスト評価者スイートを活用することもできます。

この例では、モデルが以下をどの程度うまく実行できるかを評価します：
1. 音声メッセージに関するユーザープロンプトに対して**適切な応答を生成する**
2. 高品質な応答を表す参考回答と**整合性を保つ**

## 依存関係のインストール + セットアップ

In [ ]:
# Install required packages
%pip install openai datasets pandas soundfile torch torchcodec pydub jiwer --quiet

In [ ]:
# Import libraries
from datasets import load_dataset, Audio
from openai import OpenAI
import base64
import os
import json
import time
import io
import soundfile as sf
import numpy as np
import pandas as pd

## データセット準備

Hugging Faceでホストされている[big_bench_audio](https://huggingface.co/datasets/ArtificialAnalysis/big_bench_audio)データセットを使用します。Big Bench Audioは、Big Bench Hardの質問のサブセットの音声版です。このデータセットは、音声入力をサポートするモデルの推論能力を評価するために使用できます。論理問題を説明する音声クリップ、カテゴリ、および公式回答が含まれています。

In [ ]:
dataset = load_dataset("ArtificialAnalysis/big_bench_audio")
# Ensure audio column is decoded into a dict with 'array' and 'sampling_rate'
dataset = dataset.cast_column("audio", Audio(decode=True))

関連するフィールドを抽出し、Evals APIでデータソースとして渡すためにJSON形式で配置します。入力音声データはbase64エンコードされた文字列の形式である必要があります。音声ファイル内のデータを処理し、base64に変換します。

注意：音声モデルは現在、WAV、MP3、FLAC、Opus、またはPCM16形式をサポートしています。詳細については[audio inputs](https://platform.openai.com/docs/api-reference/chat/create#chat_create-audio)を参照してください。

In [ ]:
# Audio helpers: file/array to base64
def get_base64(audio_path_or_datauri: str) -> str:
    if audio_path_or_datauri.startswith("data:"):
        # Already base64, just strip prefix
        return audio_path_or_datauri.split(",", 1)[1]
    else:
        # It's a real file path
        with open(audio_path_or_datauri, "rb") as f:
            return base64.b64encode(f.read()).decode("ascii")


def audio_to_base64(audio_val) -> str:
    """
    Accepts various Hugging Face audio representations and returns base64-encoded WAV bytes (no data: prefix).
    Handles:
      - dict or mapping-like with 'path'
      - decoded dict with 'array' and 'sampling_rate'
      - torchcodec AudioDecoder (mapping-like access via ['path'] or ['array'])
      - raw bytes
    """
    # Try to get a file path first
    try:
        path = None
        if isinstance(audio_val, dict) and "path" in audio_val:
            path = audio_val["path"]
        else:
            # Mapping-like access
            try:
                path = audio_val["path"]  # works for many decoder objects
            except Exception:
                path = getattr(audio_val, "path", None)
        if isinstance(path, str) and os.path.exists(path):
            with open(path, "rb") as f:
                return base64.b64encode(f.read()).decode("ascii")
    except Exception:
        pass

    # Fallback: use array + sampling_rate and render to WAV in-memory
    try:
        array = None
        sampling_rate = None
        try:
            array = audio_val["array"]
            sampling_rate = audio_val["sampling_rate"]
        except Exception:
            array = getattr(audio_val, "array", None)
            sampling_rate = getattr(audio_val, "sampling_rate", None)
        if array is not None and sampling_rate is not None:
            audio_np = np.array(array)
            buf = io.BytesIO()
            sf.write(buf, audio_np, int(sampling_rate), format="WAV")
            return base64.b64encode(buf.getvalue()).decode("ascii")
    except Exception:
        pass

    if isinstance(audio_val, (bytes, bytearray)):
        return base64.b64encode(audio_val).decode("ascii")

    raise ValueError("Unsupported audio value; could not convert to base64")


In [ ]:
evals_data_source = []
audio_base64 = None

# Will use the first 3 examples for testing
for example in dataset["train"].select(range(3)):
    audio_val = example["audio"]
    try:
        audio_base64 = audio_to_base64(audio_val)
    except Exception as e:
        print(f"Warning: could not encode audio for id={example['id']}: {e}")
        audio_base64 = None
    evals_data_source.append({
        "item": {
            "id": example["id"],
            "category": example["category"],
            "official_answer": example["official_answer"],
            "audio_base64": audio_base64
        }
    })


データソースリストを出力すると、各項目は以下のような形式になります：

```python
{
  "item": {
    "id": 0
    "category": "formal_fallacies"
    "official_answer": "invalid"
    "audio_base64": "UklGRjrODwBXQVZFZm10IBAAAAABAAEAIlYAAESsA..."
  }
}
```

## Eval設定

データソースとタスクが準備できたので、次にevalsを作成します。OpenAI Evals APIのドキュメントについては、[APIドキュメント](https://platform.openai.com/docs/guides/evals)をご覧ください。

In [ ]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

音声入力は大きいため、例をファイルに保存してAPIにアップロードする必要があります。

In [ ]:
# Save the examples to a file
file_name = "evals_data_source.json"
with open(file_name, "w", encoding="utf-8") as f:
    for obj in evals_data_source:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

# Upload the file to the API
file = client.files.create(
    file=open(file_name, "rb"),
    purpose="evals"
)

Evalsには2つの部分があります：「Eval」と「Run」です。「Eval」では、データの期待される構造とテスト基準（grader）を定義します。

### データソース設定

私たちが収集したデータに基づいて、データソース設定は以下の通りです：

In [ ]:
data_source_config = {
    "type": "custom",
    "item_schema": {
        "type": "object",
        "properties": {
          "id": { "type": "integer" },
          "category": { "type": "string" },
          "official_answer": { "type": "string" },
          "audio_base64": { "type": "string" }
        },
        "required": ["id", "category", "official_answer", "audio_base64"]
      },
    "include_sample_schema": True, # enables sampling
}

### テスト基準

テスト基準として、グレーダー設定を行います。この例では、公式回答とサンプリングされたモデル応答（`sample`名前空間内）を受け取り、モデル応答が公式回答と一致するかどうかに基づいて0または1のスコアを出力する`score_model`グレーダーを使用します。応答には音声とその音声のテキスト転写の両方が含まれています。グレーダーでは音声を使用します。グレーダーの詳細については、[API Grader docs](https://platform.openai.com/docs/api-reference/graders)をご覧ください。

効果的な評価を行うには、データとグレーダーの両方を適切に設定することが重要です。グレーダーのプロンプトは反復的に改良していく必要があるでしょう。

In [ ]:
grader_config = {
  "type": "score_model",
  "name": "Reference answer audio model grader",
  "model": "gpt-audio",
  "input": [
        {
            "role": "system",
            "content": 'You are a helpful assistant that evaluates audio clips to judge whether they match a provided reference answer. The audio clip is the model''s response to the question. Respond ONLY with a single JSON object matching: {"steps":[{"description":"string","conclusion":"string"}],"result":number}. Do not include any extra text. result must be a float in [0.0, 1.0].'
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": "Evaluate this audio clip to see if it reaches the same conclusion as the reference answer. Reference answer: {{item.official_answer}}",
                },
                {
                    "type": "input_audio",
                    "input_audio": {
                        "data": "{{ sample.output_audio.data }}",
                        "format": "wav",
                    },
                },
            ],
        },
    ],
          "range": [0, 1],
          "pass_threshold": 0.6,
}

代替案として、公式回答とサンプルされたモデルレスポンス（`sample`名前空間内）を受け取り、モデルレスポンスが参照回答を含んでいるかどうかに基づいて0から1の間のスコアを出力するstring_checkグレーダーを使用することもできます。レスポンスには音声とその音声のテキスト転写の両方が含まれています。グレーダーではテキスト転写を使用します。

```python
grader_config = {
  "type": "string_check",
  "name": "String check grader",
  "input": "{{sample.output_text}}",
  "reference": "{{item.official_answer}}",
  "operation": "ilike"
}
```

次に、evalオブジェクトを作成します。

In [ ]:
eval_object = client.evals.create(
        name="Audio Grading Cookbook",
        data_source_config=data_source_config,
        testing_criteria=[grader_config],
    )

## Eval実行

実行を作成するために、eval オブジェクト ID、データソース（つまり、先ほどコンパイルしたデータ）、およびモデル応答を生成するためのサンプリングに使用するチャットメッセージ入力を渡します。

この例で使用するサンプリングメッセージ入力は以下の通りです。

In [ ]:
sampling_messages = [
    {
        "role": "system",
        "content": "You are a helpful and obedient assistant that can answer questions with audio input. You will be given an audio input containing a question to answer."
    },
    {
        "role": "user",
        "type": "message",
        "content": {
            "type": "input_text",
            "text": "Answer the following question by replying with brief reasoning statements and a conclusion with a single word answer: 'valid' or 'invalid'."
        }
    },
    {
        "role": "user",
        "type": "message",
        "content": {
            "type": "input_audio",
            "input_audio": {
                "data": "{{ item.audio_base64 }}",
                "format": "wav"
            }
        }
    }]

評価実行を開始します。

In [ ]:
eval_run = client.evals.runs.create(
        name="Audio Input Eval Run",
        eval_id=eval_object.id,
        data_source={
            "type": "completions", # sample using completions API; responses API is not supported for audio inputs
            "source": {
                "type": "file_id",
                "id": file.id
            },
            "model": "gpt-audio", # model used to generate the response; check that the model you use supports audio inputs
            "sampling_params": {
                "temperature": 0.0,
            },
            "input_messages": {
                "type": "template", 
                "template": sampling_messages},
            "modalities": ["audio", "text"],
        },
    )

## 結果をポーリングして表示する

実行が完了したら、結果を確認できます。また、組織のOpenAI Evalsダッシュボードで進行状況と結果を確認することもできます。

In [ ]:
while True:
    run = client.evals.runs.retrieve(run_id=eval_run.id, eval_id=eval_object.id)
    if run.status == "completed":
        output_items = list(client.evals.runs.output_items.list(
            run_id=run.id, eval_id=eval_object.id
        ))
        df = pd.DataFrame({
                "id": [item.datasource_item["id"]for item in output_items],
                "category": [item.datasource_item["category"] for item in output_items],
                "official_answer": [item.datasource_item["official_answer"] for item in output_items],
                "model_response": [item.sample.output[0].content for item in output_items],
                "grading_results": ["passed" if item.results[0]["passed"] else "failed"
                                    for item in output_items]
            })
        display(df)
        break
    if run.status == "failed":
        print(run.error)
        break
    time.sleep(5)

### 個別の出力項目の表示

完全な出力アイテムを確認するには、以下のようにします。出力アイテムの構造は、API ドキュメントの[こちら](https://platform.openai.com/docs/api-reference/evals/run-output-item-object)で指定されています。

In [ ]:
first_item = output_items[0]

print(json.dumps(dict(first_item), indent=2, default=str))

## 結論

このクックブックでは、OpenAI Evals APIを使用してモデルへのネイティブ音声入力を評価するワークフローを説明しました。スコアモデルグレーダーを使用して音声応答を採点する方法を実演しました。

### 次のステップ
- この例をあなた自身のユースケースに変換してください。
- 大きな音声クリップがある場合は、最大8GBまでサポートする[uploads API](https://platform.openai.com/docs/api-reference/uploads/create)の使用を試してください。
- [Evalsダッシュボード](https://platform.openai.com/evaluations)に移動して、出力を可視化し、評価のパフォーマンスに関する洞察を得てください。